## Data Preprocessing for URL dataset pulled from the Mendeley website. 

### This data will be used in a final course project in CS 549 Fall 2025, San Diego State University.

Author: Jia Gapuz

1. Import the dataset

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

csv_path = Path("URL dataset.csv")

# Read with fallback encoding
try:
    df = pd.read_csv(csv_path)
except UnicodeDecodeError:
    df = pd.read_csv(csv_path, encoding="latin-1")

print(f"Loaded {len(df):,} rows")
df.head()

2. Drop exact duplicates in the dataset, print the ratio of legitimate to malicious URLS in the dataset.

In [ ]:
#log number of rows before removing duplicates
original_rows = len(df)

#drop exact duplicates
df = df.drop_duplicates()

#log current number of rows and print removed count
new_rows = len(df)
removed = original_rows - new_rows
print(f"Removed {removed} duplicate rows ({removed/original_rows:.2%} of original).")

# Count each type and compute ratio
counts = df['type'].value_counts(dropna=False)
ratio = (counts / counts.sum()).rename('ratio')
summary = pd.concat([counts.rename('count'), ratio], axis=1)

print("\nCounts and ratios by type:")
summary

Since the current dataset's ratio is unaaceptable (we defined acceptable as at least 60-40), we will pull from another dataset to reach a 50-50 if possible. 

In [ ]:
# Calculate how many more malicious entries are needed for 50-50
legit = df['type'].value_counts().get('legitimate', 0)
malicious = df['type'].value_counts().get('malicious', 0)
entries_needed = max(0, legit - malicious)
print(f"To achieve a 50-50 ratio, you need to add {entries_needed} more malicious entries.")

In [ ]:
phish_df = pd.read_csv("Phishing URLs.csv")

# Ensure we append exactly 'entries_needed' new URLs by excluding existing ones first
existing_urls = set(df['url'].astype(str)) if 'url' in df.columns else set()
candidates = phish_df["url"].astype(str).dropna().drop_duplicates()
unique_new = candidates[~candidates.isin(existing_urls)]

to_use = unique_new.head(int(entries_needed))
rows_to_append = pd.DataFrame({
    'url': to_use.values,
    'type': ['phishing'] * len(to_use)
})

#append entries
df = pd.concat([df, rows_to_append], ignore_index=True)

#turn all 'malicious' labels into 'phishing'
df['type'] = df['type'].replace({'malicious': 'phishing'})

#print summary
counts = df['type'].value_counts(dropna=False)
ratio = (counts / counts.sum()).rename('ratio')
summary = pd.concat([counts.rename('count'), ratio], axis=1)
print(summary)


3. Ensure all entries are in lowercase, relabel legitimate entries to a 0 and phishing to 1

In [ ]:
obj_cols = df.select_dtypes(include=["object"]).columns
for col in obj_cols:
    mask = df[col].notna()
    df.loc[mask, col] = df.loc[mask, col].str.strip().str.lower()

#relabel the type column to 1 or 0
if 'type' in df.columns:
    df['type'] = df['type'].map({'legitimate': 0, 'phishing': 1})

#display a few rows
df.head()

4. Use a parsing library (ie urllib.parse) to extract and append the following information:

Scheme
Subdomain
Registrable domain
Suffix
Path
Query
Fragment
Port
Username
Password
Host

In [ ]:
from urllib.parse import urlparse

# Optional robust domain parsing
try:
    import tldextract
    has_tldextract = True
except ImportError:
    has_tldextract = False


def parse_url(u: str):
    try:
        parsed = urlparse(u)
    except ValueError:
        # Handle malformed URLs (e.g., invalid IPv6)
        return {
            'scheme': '',
            'subdomain': '',
            'registrable_domain': '',
            'suffix': '',
            'path': '',
            'query': '',
            'fragment': '',
            'port': '',
            'username': '',
            'password': '',
            'host': '',
        }
    
    host = parsed.hostname or ''
    # Handle port safely - urlparse.port may raise ValueError for invalid ports
    try:
        port = parsed.port if parsed.port is not None else ''
    except ValueError:
        # If port cannot be parsed, extract it manually or set to empty
        port = ''
    username = parsed.username or ''
    password = parsed.password or ''
    scheme = parsed.scheme or ''
    path = parsed.path or ''
    query = parsed.query or ''
    fragment = parsed.fragment or ''

    # Domain parts
    if has_tldextract and host:
        ext = tldextract.extract(host)
        subdomain = ext.subdomain or ''
        registrable_domain = (ext.domain + '.' + ext.suffix) if ext.domain and ext.suffix else (ext.domain or '')
        suffix = ext.suffix or ''
    else:
        parts = host.split('.') if host else []
        suffix = parts[-1] if len(parts) >= 1 else ''
        domain = parts[-2] if len(parts) >= 2 else ''
        subdomain = '.'.join(parts[:-2]) if len(parts) >= 3 else ''
        registrable_domain = (domain + ('.' + suffix if suffix else '')) if domain else ''

    return {
        'scheme': scheme,
        'subdomain': subdomain,
        'registrable_domain': registrable_domain,
        'suffix': suffix,
        'path': path,
        'query': query,
        'fragment': fragment,
        'port': port,
        'username': username,
        'password': password,
        'host': host,
    }

components = df['url'].astype(str).fillna('').apply(parse_url)
comp_df = pd.DataFrame(list(components))

# Append columns to df (align by index)
df = pd.concat([df, comp_df], axis=1)

# Preview new columns
df[['scheme','subdomain','registrable_domain','suffix','path','query','fragment','port','username','password','host']].head()

5. Add two new columns to flag http and https entries

In [ ]:
url_series = df['url'].astype(str).fillna('')
df['is_http'] = np.where(url_series.str.startswith('http://'), 1, 0)
df['is_https'] = np.where(url_series.str.startswith('https://'), 1, 0)

df.head()

6. Count and append the following lengths:  
    a. Total length  
    b. Host length  
    c. Path length  
    d. Query length  
    e. Fragment length  

In [ ]:
# Compute and append URL length features
from urllib.parse import urlparse

# Ensure 'url' exists
if 'url' not in df.columns:
    raise KeyError("Column 'url' not found; cannot compute length features.")

url_s = df['url'].astype(str).fillna('')

# Prefer already-parsed columns if present; otherwise parse on the fly
if 'host' in df.columns:
    host_s = df['host'].astype(str).fillna('')
else:
    host_s = url_s.apply(lambda u: urlparse(u).hostname or '')

if 'path' in df.columns:
    path_s = df['path'].astype(str).fillna('')
else:
    path_s = url_s.apply(lambda u: urlparse(u).path or '')

if 'query' in df.columns:
    query_s = df['query'].astype(str).fillna('')
else:
    query_s = url_s.apply(lambda u: urlparse(u).query or '')

if 'fragment' in df.columns:
    fragment_s = df['fragment'].astype(str).fillna('')
else:
    fragment_s = url_s.apply(lambda u: urlparse(u).fragment or '')

# Add length columns
df['len_total'] = url_s.str.len()
df['len_host'] = host_s.str.len()
df['len_path'] = path_s.str.len()
df['len_query'] = query_s.str.len()
df['len_fragment'] = fragment_s.str.len()

print("Added length columns: len_total, len_host, len_path, len_query, len_fragment")

df[['len_total','len_host','len_path','len_query','len_fragment']].head()

7. Count and append the counts the following characters:  
    a. Dots  
    b. Slashes  
    c. Other special characters (ie -, _, %, @, ?, =, &)  
    d. Digits  

In [ ]:
# Count and append character occurrence features
import pandas as pd

if 'url' not in df.columns:
    raise KeyError("Column 'url' not found; cannot compute character counts.")

url_s = df['url'].astype(str).fillna('')

# Core counts
df['count_dots'] = url_s.str.count(r'\.')
df['count_slashes'] = url_s.str.count('/')
df['count_digits'] = url_s.str.count(r'\d')

# Individual special characters
df['count_hyphen'] = url_s.str.count('-')
df['count_underscore'] = url_s.str.count('_')
df['count_percent'] = url_s.str.count('%')
df['count_at'] = url_s.str.count('@')
df['count_question'] = url_s.str.count(r'\?')
df['count_equal'] = url_s.str.count('=')
df['count_ampersand'] = url_s.str.count('&')

# Aggregate special count (from the listed characters)
special_cols = [
    'count_hyphen','count_underscore','count_percent',
    'count_at','count_question','count_equal','count_ampersand'
]
df['count_special'] = df[special_cols].sum(axis=1)

# Preview of new columns
df[['count_dots','count_slashes','count_digits','count_special'] + special_cols].head()

8. Calculate and append the Shannon entropy of the entries

In [ ]:
# Calculate and append Shannon entropy of URL entries
from collections import Counter
import math

if 'url' not in df.columns:
    raise KeyError("Column 'url' not found; cannot compute entropy.")

url_s = df['url'].astype(str).fillna('')

def shannon_entropy(s: str) -> float:
    if not s:
        return 0.0
    counts = Counter(s)
    n = len(s)
    return -sum((c/n) * math.log2(c/n) for c in counts.values() if c)

# Entropy of full URL string
df['entropy_url'] = url_s.apply(shannon_entropy)

print("Added entropy column: entropy_url")

df[['entropy_url']].head()

9. Flag the following keywords  
a. Login  
b. Verify  
c. Update  
d. Secure  
e. Account  
f. Bank  
g. Paypal  
h. Free  
i. Prize  
j. Gift  
k. Confirm  
l. Win  
m. Signin  
n. Support   

In [ ]:
# Flag URLs containing any of the specified keywords
import re

if 'url' not in df.columns:
    raise KeyError("Column 'url' not found; cannot flag keywords.")

keywords = [
    'login','verify','update','secure','account','bank','paypal',
    'free','prize','gift','confirm','win','signin','support'
]
pattern = re.compile(r'(' + '|'.join(map(re.escape, keywords)) + r')', flags=re.IGNORECASE)

text = df['url'].astype(str).fillna('')
df['keyword_flag'] = text.str.contains(pattern, na=False).astype(int)

# Ensure the new column appears at the end explicitly
cols = list(df.columns)
cols.append(cols.pop(cols.index('keyword_flag')))
df = df[cols]

print(f"Total rows flagged: {df['keyword_flag'].sum():,}")

df[['url','keyword_flag']].head()

10. Flag link shorteners (ie. bit.ly, t.co, tinyurl.com, goo.gl) 

In [ ]:
# Flag common link shorteners in URLs
from urllib.parse import urlparse

# Known URL shortener registrable domains (expand as needed)
shorteners = {
    'bit.ly', 't.co', 'tinyurl.com', 'goo.gl'
}

if 'url' not in df.columns:
    raise KeyError("Column 'url' not found; cannot flag link shorteners.")

# Choose the best available field for matching
if 'registrable_domain' in df.columns:
    base = df['registrable_domain'].astype(str).str.lower()
elif 'host' in df.columns:
    base = df['host'].astype(str).str.lower()
else:
    base = df['url'].astype(str).fillna('').apply(lambda u: (urlparse(u).hostname or '').lower())

# Create flag column
is_short = base.isin(shorteners).astype(int)
df['is_shortened'] = is_short

# Move the new column to the end explicitly
cols = list(df.columns)
cols.append(cols.pop(cols.index('is_shortened')))
df = df[cols]

print(f"Total shorteners flagged: {int(is_short.sum()):,}")

# Preview
preview_cols = ['url']
if 'registrable_domain' in df.columns:
    preview_cols.append('registrable_domain')
elif 'host' in df.columns:
    preview_cols.append('host')
preview_cols.append('is_shortened')
df[preview_cols].head()

11. Flag all domains that are purely IP numbers (ie http://101.10.1.101)

In [ ]:
# Flag hosts that are pure IP addresses (IPv4 or IPv6)
from urllib.parse import urlparse
import ipaddress

if 'url' not in df.columns:
    raise KeyError("Column 'url' not found; cannot flag IP hosts.")

# Prefer previously parsed host if available
if 'host' in df.columns:
    host_s = df['host'].astype(str).fillna('')
else:
    host_s = df['url'].astype(str).fillna('').apply(lambda u: (urlparse(u).hostname or ''))


def is_ip_host(h: str) -> int:
    try:
        ipaddress.ip_address(h)
        return 1
    except ValueError:
        return 0

# Create flag
df['is_ip_host'] = host_s.apply(is_ip_host).astype(int)

# Move the column to the end explicitly
cols = list(df.columns)
cols.append(cols.pop(cols.index('is_ip_host')))
df = df[cols]

print(f"Total IP hosts flagged: {int(df['is_ip_host'].sum()):,}")

# Preview
preview_cols = ['url']
if 'host' in df.columns:
    preview_cols.append('host')
preview_cols.append('is_ip_host')
df[preview_cols].head()

12. Save the anotated dataset

In [ ]:
# Save the processed DataFrame to CSV
from pathlib import Path

#print out df preview
df.head()

output_path = Path("../processed_urls.csv")
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"Saved {len(df):,} rows x {df.shape[1]} columns to {output_path.resolve()}")